In [1]:
from TSPLIPLoader import TPSLIBLoader

berlin52 = TPSLIBLoader('berlin52')
burma14 = TPSLIBLoader('burma14')


berlin52.nodes

best_cost = berlin52.get_solution

In [2]:
berlin52.get_solution

7542

In [ ]:
from algorithms.HSFFPKO import HSFFPKO, universal_tsp_wrapper
from algorithms.EHO import elephant_herd_optimization
from algorithms.ACO import ant_colony
from algorithms.SA import simulated_annealing
from algorithms.SAI import simulated_annealing_improved
from algorithms.GWO import GreyWolfTSP
from algorithms.PSO import hybrid_pso_tsp
from algorithms.GA import genetic_alg 

num_nodes = len(berlin52.nodes)
graph = berlin52.to_graph()
matrix = berlin52.to_matrix()

Shape: (10, 10)
Sample: [ 0. 20. 13. 21. 16.]...


In [4]:
import time 
import numpy as np

data = {
    'HSFFPKO': [],
    'EHO': [],
    'ACO': [],
    'SA' : [],
    'SAI': [],
    'GWO': [],
    'PSO': [],
    'GA': []
    
}

N = 10
for _ in range(N):
    start = time.perf_counter()
    output = HSFFPKO(
        Popsize=50,
        Maxiteration=500,
        LB=np.zeros(num_nodes),
        UB=np.ones(num_nodes) * 20,
        Dim=num_nodes,
        Fobj=universal_tsp_wrapper(matrix),  # MATRIX
        scouts_rate=0.15,
        num_flocks=8
    )[0] 
    end = time.perf_counter()
    data['HSFFPKO'].append([output,end - start])
    
    start = time.perf_counter()
    output = elephant_herd_optimization(graph)[1]
    end = time.perf_counter()
    data['EHO'].append([output,end - start])

    start = time.perf_counter()
    output = ant_colony(graph)[1]
    end = time.perf_counter()
    data['ACO'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing(graph, temperature=1000, alpha=0.995, iterations=1000)[1]
    end = time.perf_counter()
    data['SA'].append([output,end - start])

    start = time.perf_counter()
    output = simulated_annealing_improved(graph, iter_per_temp=100)[0]
    end = time.perf_counter()
    data['SAI'].append([output,end - start])

    gwo = GreyWolfTSP(graph)
    start = time.perf_counter()
    output = gwo.optimize()[1]
    end = time.perf_counter()
    data['GWO'].append([output,end - start])

    start = time.perf_counter()
    output = hybrid_pso_tsp(graph)[1]
    end = time.perf_counter()
    data['PSO'].append([output,end - start])

    start = time.perf_counter()
    output = genetic_alg(graph, pop_size=100, mutation_rate=0.01, generations=50)[1]
    end = time.perf_counter()
    data['GA'].append([output,end - start])




In [5]:
import pandas as pd

cost_dict = {k: [x[0] for x in v] for k, v in data.items()}
df_cost = pd.DataFrame(cost_dict)

runtime_dict = {k: [x[1] for x in v] for k, v in data.items()}
df_runtime = pd.DataFrame(runtime_dict)

# Get the relative error
def relative_error(costs, best_cost):
    return (costs - best_cost) / best_cost

rel_error_dict = {k: relative_error(np.array(v), best_cost) for k, v in cost_dict.items()}
df_rel_error = pd.DataFrame(rel_error_dict)

cols = ['HSFFPKO', 'EHO', 'ACO', 'SA', 'SAI', 'GWO', 'PSO', 'GA']

df = pd.DataFrame(
    np.vstack([df_cost.mean(), df_runtime.mean(), df_cost.min(), df_rel_error.mean() * 100, df_rel_error.min() * 100]),
    columns=cols,
    index=['Avg Cost', 'Avg Runtime', 'Min Cost', 'Avg Rel Error', 'Min Rel Error']
).transpose()

df.sort_values(by='Avg Cost', inplace=True)

df

,Avg Cost,Avg Runtime,Min Cost,Avg Rel Error,Min Rel Error
GA,7542.0,169.523411,7542.0,0.000000,0.000000
SAI,7718.4,2.592760,7542.0,2.338902,0.000000
ACO,7720.3,29.054481,7674.0,2.364094,1.750199
EHO,9281.8,0.501700,8718.0,23.068152,15.592681
PSO,10725.1,6.616420,10110.0,42.204985,34.049324
SA,13958.0,0.011460,12686.0,85.070273,68.204720
HSFFPKO,15888.5,1.842191,14489.0,110.666932,92.110846
GWO,19981.5,1.812700,13765.0,164.936356,82.511270


In [6]:
df_cost

,HSFFPKO,EHO,ACO,SA,SAI,GWO,PSO,GA
0,14489.0,9535.0,7706.0,13983,7673,17767,11184,7542
1,15744.0,9637.0,7768.0,13738,7874,21283,10270,7542
2,16600.0,9259.0,7824.0,13622,7954,13765,10110,7542
3,16788.0,9724.0,7680.0,15350,7755,23755,10353,7542
4,15226.0,8910.0,7679.0,12927,7542,23286,10350,7542
5,17244.0,9468.0,7838.0,16127,7542,22966,10166,7542
6,15263.0,9140.0,7679.0,13386,7755,14011,10771,7542
7,16365.0,8718.0,7680.0,13679,7542,18063,11487,7542
8,15801.0,9424.0,7674.0,14082,8005,22122,10750,7542
9,15365.0,9003.0,7675.0,12686,7542,22797,11810,7542


In [7]:
df_runtime

,HSFFPKO,EHO,ACO,SA,SAI,GWO,PSO,GA
0,2.459631,0.682659,30.478190,0.012671,2.678372,1.776740,7.006674,146.256023
1,1.823557,0.487186,29.130295,0.011453,2.629121,1.802493,6.911400,161.512546
2,1.900112,0.525217,29.176852,0.011739,2.623402,1.880163,6.487887,231.213820
3,1.737413,0.477571,28.855158,0.011201,2.604979,1.823235,6.489773,159.535533
4,1.679867,0.459938,27.692310,0.010854,2.393911,1.710907,6.189883,146.281902
5,1.821933,0.480720,29.213952,0.011027,2.545470,1.795473,6.883630,135.854298
6,1.709957,0.464396,28.975201,0.011009,2.476486,1.758930,6.387655,154.574640
7,1.770397,0.474985,28.920929,0.012482,2.900808,2.011009,6.550156,154.478454
8,1.751401,0.472243,29.412729,0.011120,2.538454,1.774218,6.639757,200.546423
9,1.767644,0.492083,28.689197,0.011040,2.536598,1.793837,6.617388,204.980469
